In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import tushare as ts

In [4]:
# 拉取数据
pro = ts.pro_api()
df_csi300 = pro.index_daily(**{
    "ts_code": "000300.SH",
    "trade_date": "",
    "start_date": 20200101,
    "end_date": 20251001,
    "limit": "",
    "offset": ""
}, fields=[
    "ts_code",
    "trade_date",
    "close",
    "open",
    "high",
    "low",
    "pre_close",
    "change",
    "pct_chg",
    "vol",
    "amount"
])

In [6]:
try:
    df_csi= pro.index_weight(index_code='000300.SH',trade_date='20250930')
    
    # 3. 从返回的DataFrame中提取股票代码列表
    # 成分股代码在 'con_code' 这一列
    csi300_stock_list = df_csi['con_code'].tolist()
    
    print(f"成功获取到 {len(csi300_stock_list)} 只沪深300成分股。")
    print("部分成分股代码示例:")
    print(csi300_stock_list[:10])

except Exception as e:
    print(f"获取数据失败: {e}")

成功获取到 300 只沪深300成分股。
部分成分股代码示例:
['300750.SZ', '600519.SH', '601318.SH', '600036.SH', '601899.SH', '300502.SZ', '300308.SZ', '000333.SZ', '300059.SZ', '600900.SH']


In [ ]:
all_stock_data = pd.read_csv('data/all_stock_data_20200101_20250922.csv')

In [10]:
all_stock_data['free_share']/all_stock_data['float_share']

0          0.686073
1          0.788235
2          0.647688
3          0.333619
4          0.426186
             ...   
6687291    0.726919
6687292    1.000000
6687293    0.805922
6687294    0.595411
6687295    0.432493
Length: 6687296, dtype: float64

In [9]:
all_stock_data['turnover_rate']/all_stock_data['turnover_rate_f']

0          0.686080
1          0.788227
2          0.647685
3          0.333621
4          0.426190
             ...   
6687291    0.726879
6687292    1.000000
6687293    0.805909
6687294    0.595422
6687295    0.432495
Length: 6687296, dtype: float64

In [17]:
np.min(all_stock_data['turnover_rate'])

0.0001